IMPORTUJEMY POTRZEBNE RZECZY

In [10]:
# Importujemy potrzebne biblioteki
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
import pickle
import os
import ast
from sklearn.feature_extraction import DictVectorizer

Uczymy model wedle rozbicia na 70% danych treningowych

In [ ]:
# Ścieżki do danych
train_texts_path = '../data/70_30/train_texts_bow.csv'
train_labels_path = '../data/70_30/train_labels.csv'

# Wczytujemy dane
X_train_raw = pd.read_csv(train_texts_path, index_col=0)
y_train = pd.read_csv(train_labels_path, index_col=0).squeeze()

print("Dane treningowe wczytane poprawnie.")
print(f"X_train_raw shape: {X_train_raw.shape}")
print(f"y_train shape: {y_train.shape}")

print("Rozkład klas w zbiorze treningowym:")
print(y_train.value_counts(normalize=True))

# Wyrównujemy indeksy
X_train_raw, y_train = X_train_raw.align(y_train, join='inner', axis=0)

# Zamieniamy stringi na słowniki
X_train_dicts = X_train_raw.iloc[:, 0].apply(ast.literal_eval)

# Konwersja na macierz
vectorizer = DictVectorizer(sparse=True)
X_train = vectorizer.fit_transform(X_train_dicts)

print("Dane zostały przekonwertowane na macierz numeryczną.")

# Trenujemy model Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

print("Model został wytrenowany.")

# Pakujemy model i vectorizer razem
model_package = {
    'vectorizer': vectorizer,
    'model': model
}

# Zapisujemy do jednego pliku
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'BoW_70_30.pkl')

with open(model_path, 'wb') as file:
    pickle.dump(model_package, file)

print(f"Model (wraz z vectorizerem) zapisano w: {model_path}")


Dane treningowe wczytane poprawnie.
X_train_raw shape: (69997, 1)
y_train shape: (70000,)
Rozkład klas w zbiorze treningowym:
stars
5    0.443514
4    0.253043
3    0.113557
1    0.109243
2    0.080643
Name: proportion, dtype: float64
Dane zostały przekonwertowane na macierz numeryczną.
Model został wytrenowany.
Model (wraz z vectorizerem) zapisano w: ../models\BoW_70_30.pkl


Ćwiczymy model dla 80% danych testowych

In [14]:
# Ścieżki do danych
train_texts_path = '../data/80_20/train_texts_bow.csv'
train_labels_path = '../data/80_20/train_labels.csv'

# Wczytujemy dane
X_train_raw = pd.read_csv(train_texts_path, index_col=0)
y_train = pd.read_csv(train_labels_path, index_col=0).squeeze()

print("Dane treningowe wczytane poprawnie.")
print(f"X_train_raw shape: {X_train_raw.shape}")
print(f"y_train shape: {y_train.shape}")

print("Rozkład klas w zbiorze treningowym:")
print(y_train.value_counts(normalize=True))

# Wyrównujemy indeksy
X_train_raw, y_train = X_train_raw.align(y_train, join='inner', axis=0)

# Zamieniamy stringi na słowniki
X_train_dicts = X_train_raw.iloc[:, 0].apply(ast.literal_eval)

# Konwersja na macierz
vectorizer = DictVectorizer(sparse=True)
X_train = vectorizer.fit_transform(X_train_dicts)

print("Dane zostały przekonwertowane na macierz numeryczną.")

# Trenujemy model Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

print("Model został wytrenowany.")

# Pakujemy model i vectorizer razem
model_package = {
    'vectorizer': vectorizer,
    'model': model
}

# Zapisujemy do jednego pliku
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'BoW_80_20.pkl')

with open(model_path, 'wb') as file:
    pickle.dump(model_package, file)

print(f"Model (wraz z vectorizerem) zapisano w: {model_path}")

Dane treningowe wczytane poprawnie.
X_train_raw shape: (79997, 1)
y_train shape: (80000,)
Rozkład klas w zbiorze treningowym:
stars
5    0.444012
4    0.253063
3    0.113587
1    0.109137
2    0.080200
Name: proportion, dtype: float64
Dane zostały przekonwertowane na macierz numeryczną.
Model został wytrenowany.
Model (wraz z vectorizerem) zapisano w: ../models\BoW_80_20.pkl


Model może mieć tendencje do zgadywania dominujących klas, ich rozkład prezentuje się następująco

5 gwiazdek: 44.35%
4 gwiazdki: 25.30%
3 gwiazdki: 11.35%
1 gwiazdka: 10.92%
2 gwiazdki: 8.06%

Redukcja do 3 klas pozytywna, neutralna, negatywny

In [15]:
# Ścieżki do danych
train_texts_path = '../data/70_30/train_texts_bow.csv'
train_labels_path = '../data/70_30/train_labels.csv'

# Wczytujemy dane
X_train_raw = pd.read_csv(train_texts_path, index_col=0)
y_train = pd.read_csv(train_labels_path, index_col=0).squeeze()

print("Dane treningowe wczytane poprawnie.")
print(f"X_train_raw shape: {X_train_raw.shape}")
print(f"y_train shape: {y_train.shape}")

# Rozkład klas przed mapowaniem (kontrola)
print("Rozkład klas w zbiorze treningowym (przed mapowaniem):")
print(y_train.value_counts(normalize=True))

# Mapujemy gwiazdki na klasy sentymentu
def map_sentiment(star_rating):
    if star_rating in [4, 5]:
        return 'Pozytywna'
    elif star_rating == 3:
        return 'Neutralna'
    else:
        return 'Negatywna'

y_train = y_train.map(map_sentiment)

print("Rozkład klas w zbiorze treningowym (po mapowaniu):")
print(y_train.value_counts(normalize=True))

# Diagnoza indeksów przed align
print(f"Indeksy X_train_raw: {list(X_train_raw.index[:5])} ...")
print(f"Indeksy y_train: {list(y_train.index[:5])} ...")

# Wyrównujemy indeksy
X_train_raw, y_train = X_train_raw.align(y_train, join='inner', axis=0)

print(f"Po align: X_train_raw shape: {X_train_raw.shape}, y_train shape: {y_train.shape}")

# Zamieniamy stringi na słowniki
X_train_dicts = X_train_raw.iloc[:, 0].apply(ast.literal_eval)

# Konwersja na macierz
vectorizer = DictVectorizer(sparse=True)
X_train = vectorizer.fit_transform(X_train_dicts)

print("Dane zostały przekonwertowane na macierz numeryczną.")

# Tworzymy i trenujemy model Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

print("Model został wytrenowany.")

# Pakujemy model i vectorizer razem w jeden plik
model_package = {
    'vectorizer': vectorizer,
    'model': model
}

# Zapisujemy do jednego pliku
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'BoW_70_30_PNN.pkl')

with open(model_path, 'wb') as file:
    pickle.dump(model_package, file)

print(f"Model zapisano w: {model_path}")

Dane treningowe wczytane poprawnie.
X_train_raw shape: (69997, 1)
y_train shape: (70000,)
Rozkład klas w zbiorze treningowym (przed mapowaniem):
stars
5    0.443514
4    0.253043
3    0.113557
1    0.109243
2    0.080643
Name: proportion, dtype: float64
Rozkład klas w zbiorze treningowym (po mapowaniu):
stars
Pozytywna    0.696557
Negatywna    0.189886
Neutralna    0.113557
Name: proportion, dtype: float64
Indeksy X_train_raw: [76513, 60406, 27322, 53699, 65412] ...
Indeksy y_train: [76513, 60406, 27322, 53699, 65412] ...
Po align: X_train_raw shape: (69997, 1), y_train shape: (69997,)
Dane zostały przekonwertowane na macierz numeryczną.
Model został wytrenowany.
Model zapisano w: ../models\BoW_70_30_PNN.pkl


In [16]:
# Ścieżki do danych
train_texts_path = '../data/80_20/train_texts_bow.csv'
train_labels_path = '../data/80_20/train_labels.csv'

# Wczytujemy dane
X_train_raw = pd.read_csv(train_texts_path, index_col=0)
y_train = pd.read_csv(train_labels_path, index_col=0).squeeze()

print("Dane treningowe wczytane poprawnie.")
print(f"X_train_raw shape: {X_train_raw.shape}")
print(f"y_train shape: {y_train.shape}")

# Rozkład klas przed mapowaniem (kontrola)
print("Rozkład klas w zbiorze treningowym (przed mapowaniem):")
print(y_train.value_counts(normalize=True))

# Mapujemy gwiazdki na klasy sentymentu
def map_sentiment(star_rating):
    if star_rating in [4, 5]:
        return 'Pozytywna'
    elif star_rating == 3:
        return 'Neutralna'
    else:
        return 'Negatywna'

y_train = y_train.map(map_sentiment)

print("Rozkład klas w zbiorze treningowym (po mapowaniu):")
print(y_train.value_counts(normalize=True))

# Diagnoza indeksów przed align
print(f"Indeksy X_train_raw: {list(X_train_raw.index[:5])} ...")
print(f"Indeksy y_train: {list(y_train.index[:5])} ...")

# Wyrównujemy indeksy
X_train_raw, y_train = X_train_raw.align(y_train, join='inner', axis=0)

print(f"Po align: X_train_raw shape: {X_train_raw.shape}, y_train shape: {y_train.shape}")

# Zamieniamy stringi na słowniki
X_train_dicts = X_train_raw.iloc[:, 0].apply(ast.literal_eval)

# Konwersja na macierz
vectorizer = DictVectorizer(sparse=True)
X_train = vectorizer.fit_transform(X_train_dicts)

print("Dane zostały przekonwertowane na macierz numeryczną.")

# Tworzymy i trenujemy model Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

print("Model został wytrenowany.")

# Pakujemy model i vectorizer razem w jeden plik
model_package = {
    'vectorizer': vectorizer,
    'model': model
}

# Zapisujemy do jednego pliku
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'BoW_80_20_PNN.pkl')

with open(model_path, 'wb') as file:
    pickle.dump(model_package, file)

print(f"Model zapisano w: {model_path}")

Dane treningowe wczytane poprawnie.
X_train_raw shape: (79997, 1)
y_train shape: (80000,)
Rozkład klas w zbiorze treningowym (przed mapowaniem):
stars
5    0.444012
4    0.253063
3    0.113587
1    0.109137
2    0.080200
Name: proportion, dtype: float64
Rozkład klas w zbiorze treningowym (po mapowaniu):
stars
Pozytywna    0.697075
Negatywna    0.189337
Neutralna    0.113587
Name: proportion, dtype: float64
Indeksy X_train_raw: [75220, 48955, 44966, 13568, 92727] ...
Indeksy y_train: [75220, 48955, 44966, 13568, 92727] ...
Po align: X_train_raw shape: (79997, 1), y_train shape: (79997,)
Dane zostały przekonwertowane na macierz numeryczną.
Model został wytrenowany.
Model zapisano w: ../models\BoW_80_20_PNN.pkl
